# Fine Tune GPT-4o-mini

1. Create a dataset of customer service questions and answers
2. Fine tune GPT-4o-mini on the dataset
3. Evaluate the fine tuned model
4. Compare the fine tuned model to the base model


### Model Training

In [2]:
from src import FineTuner

fine_tune = FineTuner()

In [ ]:
file_id = fine_tune.upload_training_file('data/training_data.jsonl')

In [ ]:
job_id = fine_tune.create_job(training_file=file_id, model='gpt-4o-mini-2024-07-18')

In [ ]:
import time

status = fine_tune.get_job_status(job_id)

print(status)

# while status.status != 'succeeded':
#     status = fine_tune.get_job_status(job_id)
#     print(f"Job status: {status.status}")
#     time.sleep(10)

In [ ]:
fine_tune.wait_for_job(job_id)

In [ ]:
df = fine_tune.get_training_metrics(job_id)

fine_tune.plot_training_metrics(job_id)

In [ ]:
from models import Models
import pandas as pd
from tqdm import tqdm

test_questions = [
    "I'm being charged by a merchant located overseas, but I've never visited or shopped there. Can I dispute these charges?",
    "My purchase arrived damaged, and the seller won't offer a replacement. Should I dispute the transaction?",
    "I found two identical charges for one service in my statement. What are my options to dispute the duplicate?",
    "I canceled a flight, but the airline still charged me. How do I begin a dispute?",
    "My card details were used to pay for someone else's subscription. How can I report and dispute this?",
    "The merchant promised a refund but hasn't issued it yet. Should I start a dispute now?",
    "I ordered an item online, but it arrived weeks late and in poor condition. Do I have grounds for a dispute?",
    "My credit card shows repeated small charges I didn't authorize. How do I handle these?",
    "I received a completely different product than what was advertised. Should I dispute the charge immediately?",
    "The merchant says the transaction is valid, but I don't recognize it at all. Can I still dispute it?",
]

df = pd.DataFrame(test_questions, columns=["question"])

models = Models()

base_responses = []
ft_responses = []

for question in tqdm(test_questions, desc="Processing questions"):
    base_response = models.customer_service_chat(
        user = question, 
        model="gpt-4o-mini"
    )
    ft_response = models.customer_service_chat(
        user = question, 
        model="ft:gpt-4o-mini-2024-07-18:capital-on-tap:v1:Aj5rXiEu"
    )

    base_responses.append(base_response)
    ft_responses.append(ft_response)

df["base_response"] = base_responses
df["ft_response"] = ft_responses

# Set display options for better readability
pd.set_option('display.max_colwidth', 500)
